## Metoda eliminacji Gaussa

Rozwiązanie układu równań liniowych metodą eliminacji Gaussa polega na przekształceniu macierzy współczynników układu równań na macierz trójkątną górną, a następnie na rozwiązaniu układu równań za pomocą metody podstawiania wstecz. 

Jest to jedna z najbardziej podstawowych metod rozwiązywania układów równań liniowych, wychodząca poza sytuacje gdzie liczba niewiadomych jest różna od liczby równań.

In [2]:
from sympy import Matrix, symbols, latex
from IPython.display import display, Markdown

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None, display_mode="table"):
        self.matrix = Matrix(matrix)
        self.operations = []
        self.display_mode = display_mode

        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("Liczba etykiet kolumn musi być równa liczbie kolumn w macierzy.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]

        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("Liczba wyrazów wolnych musi być równa liczbie wierszy w macierzy.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]

        display(Markdown("**Początkowe równania:**"))
        self.display_matrix()

    def set_display_mode(self, mode):
        if mode not in ["table", "equations"]:
            raise ValueError("Tryb wyświetlania musi być 'table' lub 'equations'.")
        self.display_mode = mode

    def display_matrix(self):
        if self.display_mode == "table":
            self._display_table()
        elif self.display_mode == "equations":
            self._display_equations()

    def _display_table(self):
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Wyraz wolny"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            row.append(str(self.free_terms[i]))
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def _display_equations(self):
        equations = []
        for i in range(self.matrix.rows):
            terms = []
            for j in range(self.matrix.cols):
                coef = self.matrix[i, j]
                var = symbols(self.column_labels[j])
                if coef > 0:
                    terms.append(f"+ {latex(coef)}{latex(var)}")
                elif coef < 0:
                    terms.append(f"- {latex(abs(coef))}{latex(var)}")
            lhs = " ".join(terms).lstrip("+ ")
            rhs = latex(self.free_terms[i])
            equations.append(f"{lhs} = {rhs}")
        display(Markdown("Układ równań:"))
        display(Markdown(f"\\begin{{aligned}} {self._equations_to_latex(equations)} \\end{{aligned}}"))

    def _equations_to_latex(self, equations):
        return " \\\\ ".join(equations)

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])
        self.free_terms[target_idx] += coefficient * self.free_terms[source_idx]
        operation_str = f"w{target_row} = w{target_row} + {coefficient}*w{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)
        self.free_terms[row_idx] *= coefficient
        operation_str = f"w{row} = {coefficient}*w{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)
        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]
        operation_str = f"Zamiana w{row1} <-> w{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Numer wiersza musi być liczbą całkowitą.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Numer wiersza musi być w zakresie od 1 do {self.matrix.rows}.")
        return row - 1

    def print_operations(self):
        display(Markdown("**Wykonane operacje:**"))
        for op in self.operations:
            print(op)

# Przykład użycia:
m = SymbolicMatrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 12], display_mode="table")


**Początkowe równania:**

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 | 10 |
| 4 | 5 | 6 | 11 | 11 |
| 7 | 8 | 9 | 12 | 12 |


In [3]:
m.add_row(2, 1, -4)

**Operacja:** w2 = w2 + -4*w1

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 | 10 |
| 0 | -3 | -6 | -29 | -29 |
| 7 | 8 | 9 | 12 | 12 |


In [4]:
m.add_row(3, 1, -7)

**Operacja:** w3 = w3 + -7*w1

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 | 10 |
| 0 | -3 | -6 | -29 | -29 |
| 0 | -6 | -12 | -58 | -58 |


In [5]:
m.add_row(3, 2, -2)

**Operacja:** w3 = w3 + -2*w2

| x1 | x2 | x3 | Wyraz wolny |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 10 | 10 |
| 0 | -3 | -6 | -29 | -29 |
| 0 | 0 | 0 | 0 | 0 |


In [7]:
import sympy as sp

# Імпорт функцій Eq та symbols
from sympy import Eq, symbols

# Definicja symboli
x1, x2, x3 = symbols('x1 x2 x3')

# Definicja równań
eq1 = Eq(x1 + 2*x2 + 3*x3, 10)
eq2 = Eq(-3*x2 - 6*x3, -29)
eq3 = Eq(3*x3, 6) 

# Wyświetlamy równania
print("Równanie 1:")
print(eq1)

print("Równanie 2:")
print(eq2)

print("Równanie 3:")
print(eq3)


Równanie 1:
Eq(x1 + 2*x2 + 3*x3, 10)
Równanie 2:
Eq(-3*x2 - 6*x3, -29)
Równanie 3:
Eq(3*x3, 6)


In [8]:
sol_x3 = sp.solve(eq3, x3)[0]
sol_x3

2

In [9]:
# podstawiamy rozwiązanie x3 do równania 2
eq2 = eq2.subs(x3, sol_x3)
# rozwiązujemy równanie 2
sol_x2 = sp.solve(eq2, x2)[0]
sol_x2

17/3

In [10]:
# podstawiamy rozwiązania x2 i x3 do równania 1
eq1 = eq1.subs({x2: sol_x2, x3: sol_x3})
# rozwiązujemy równanie 1
sol_x1 = sp.solve(eq1, x1)[0]
sol_x1

-22/3

---

## Zadania dla studentów

$$
\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.
$$

### Zadanie 1

Rozwiążmy układ równań:
$$ \begin{cases} 
x + 2y - 2z = 4 \\
2x + y + z = 0 \\
3x + 2y + z = 1 
\end{cases} $$

**Metoda eliminacji:**

1. Eliminujemy $ z $ z pierwszego i drugiego równania:
$$ (x + 2y - 2z) + 2(2x + y + z) = 4 + 0 $$
$$ x + 2y - 2z + 4x + 2y + 2z = 4 $$
$$ 5x + 4y = 4 $$

2. Eliminujemy $ z $ z pierwszego i trzeciego równania:
$$ 3(x + 2y - 2z) + (3x + 2y + z) = 3 \cdot 4 + 1 $$
$$ 3x + 6y - 6z + 3x + 2y + z = 13 $$
$$ 6x + 8y - 5z = 13 $$

3. Rozwiązujemy układ równań:
$$ \begin{cases} 
5x + 4y = 4 \\
6x + 8y - 5z = 13 
\end{cases} $$

4. Eliminujemy $ y $ z pierwszego równania:
$$ 2(5x + 4y) = 2 \cdot 4 $$
$$ 10x + 8y = 8 $$

5. Odejmujemy drugie równanie od pierwszego:
$$ 10x + 8y - (6x + 8y - 5z) = 8 - 13 $$
$$ 4x + 5z = -5 $$
$$ x = \frac{-5 - 5z}{4} $$

6. Podstawiamy $ x $ do pierwszego równania:
$$ 5\left(\frac{-5 - 5z}{4}\right) + 4y = 4 $$
$$ \frac{-25 - 25z}{4} + 4y = 4 $$
$$ -25 - 25z + 16y = 16 $$
$$ 16y = 41 + 25z $$
$$ y = \frac{41 + 25z}{16} $$

7. Podstawiamy $ x $ i $ y $ do drugiego równania:
$$ 2\left(\frac{-5 - 5z}{4}\right) + \frac{41 + 25z}{16} + z = 0 $$
$$ \frac{-10 - 10z}{4} + \frac{41 + 25z}{16} + z = 0 $$
$$ \frac{-40 - 40z + 41 + 25z + 16z}{16} = 0 $$
$$ \frac{1 + z}{16} = 0 $$
$$ z = -1 $$

8. Podstawiamy $ z $ do wyrażeń na $ x $ i $ y $:
$$ x = \frac{-5 - 5(-1)}{4} $$
$$ x = \frac{-5 + 5}{4} $$
$$ x = 0 $$

$$ y = \frac{41 + 25(-1)}{16} $$
$$ y = \frac{41 - 25}{16} $$
$$ y = \frac{16}{16} $$
$$ y = 1 $$

Rozwiązanie:
$$ x = 0, \quad y = 1, \quad z = -1 $$

### Zadanie 2

Rozwiążmy układ równań:
$$ \begin{cases} 
x + y + z - t = 2 \\
2x + y + z = 3 \\
-x + z - t = 0 \\
3x + 2y - z + 2t = -1 
\end{cases} $$

**Metoda eliminacji:**

1. Eliminujemy $ t $ z pierwszego i trzeciego równania:
$$ (x + y + z - t) + (-x + z - t) = 2 + 0 $$
$$ x + y + z - t - x + z - t = 2 $$
$$ y + 2z - 2t = 2 $$

2. Eliminujemy $ t $ z pierwszego i czwartego równania:
$$ 3(x + y + z - t) + (3x + 2y - z + 2t) = 3 \cdot 2 + (-1) $$
$$ 3x + 3y + 3z - 3t + 3x + 2y - z + 2t = 5 $$
$$ 6x + 5y + 2z - t = 5 $$

3. Rozwiązujemy układ równań:
$$ \begin{cases} 
y + 2z - 2t = 2 \\
6x + 5y + 2z - t = 5 
\end{cases} $$

4. Eliminujemy $ y $ z pierwszego równania:
$$ 5(y + 2z - 2t) = 5 \cdot 2 $$
$$ 5y + 10z - 10t = 10 $$

5. Odejmujemy drugie równanie od pierwszego:
$$ 5y + 10z - 10t - (6x + 5y + 2z - t) = 10 - 5 $$
$$ 10z - 10t - 6x - 2z + t = 5 $$
$$ 8z - 9t - 6x = 5 $$
$$ x = \frac{5 - 8z + 9t}{6} $$

6. Podstawiamy $ x $ do pierwszego równania:
$$ 2\left(\frac{5 - 8z + 9t}{6}\right) + y + z = 3 $$
$$ \frac{10 - 16z + 18t}{6} + y + z = 3 $$
$$ 10 - 16z + 18t + 6y + 6z = 18 $$
$$ 6y - 10z + 18t = 8 $$
$$ y = \frac{8 + 10z - 18t}{6} $$

7. Podstawiamy $ x $ i $ y $ do drugiego równania:
$$ -\left(\frac{5 - 8z + 9t}{6}\right) + z - t = 0 $$
$$ -\frac{5 - 8z + 9t}{6} + z - t = 0 $$
$$ -5 + 8z - 9t + 6z - 6t = 0 $$
$$ 14z - 15t = 5 $$
$$ z = \frac{5 + 15t}{14} $$

8. Podstawiamy $ z $ do wyrażeń na $ x $ i $ y $:
$$ x = \frac{5 - 8\left(\frac{5 + 15t}{14}\right) + 9t}{6} $$
$$ x = \frac{5 - \frac{40 + 120t}{14} + 9t}{6} $$
$$ x = \frac{5 - \frac{40}{14} - \frac{120t}{14} + 9t}{6} $$
$$ x = \frac{5 - \frac{20}{7} - \frac{60t}{7} + 9t}{6} $$
$$ x = \frac{5 - \frac{20}{7} + \frac{3t}{7}}{6} $$

$$ y = \frac{8 + 10\left(\frac{5 + 15t}{14}\right) - 18t}{6} $$
$$ y = \frac{8 + \frac{50 + 150t}{14} - 18t}{6} $$
$$ y = \frac{8 + \frac{50}{14} + \frac{150t}{14} - 18t}{6} $$
$$ y = \frac{8 + \frac{25}{7} + \frac{75t}{7} - 18t}{6} $$
$$ y = \frac{8 + \frac{25}{7} + \frac{75t}{7} - 18t}{6} $$

Rozwiązanie:
$$ x = \frac{5 - \frac{20}{7} + \frac{3t}{7}}{6}, \quad y = \frac{8 + \frac{25}{7} + \frac{75t}{7} - 18t}{6}, \quad z = \frac{5 + 15t}{14} $$

### Zadanie 3

Rozwiążmy układ równań:
$$ \begin{cases} 
x + y - z - t = 0 \\
2x + 3y - 2z + t = 4 \\
3x + 5z = 0 \\
-x + y - 3z + 2t = 3 
\end{cases} $$

**Metoda eliminacji:**

1. Eliminujemy $ t $ z pierwszego i trzeciego równania:
$$ (x + y - z - t) + (-x + y - 3z + 2t) = 0 + 3 $$
$$ x + y - z - t - x + y - 3z + 2t = 3 $$
$$ 2y - 4z + t = 3 $$

2. Eliminujemy $ t $ z pierwszego i czwartego równania:
$$ 3(x + y - z - t) + (3x + 5z) = 3 \cdot 0 + 0 $$
$$ 3x + 3y - 3z - 3t + 3x + 5z = 0 $$
$$ 6x + 3y + 2z - 3t = 0 $$

3. Rozwiązujemy układ równań:
$$ \begin{cases} 
2y - 4z + t = 3 \\
6x + 3y + 2z - 3t = 0 
\end{cases} $$

4. Eliminujemy $ y $ z pierwszego równania:
$$ 3(2y - 4z + t) = 3 \cdot 3 $$
$$ 6y - 12z + 3t = 9 $$

5. Odejmujemy drugie równanie od pierwszego:
$$ 6y - 12z + 3t - (6x + 3y + 2z - 3t) = 9 - 0 $$
$$ 3y - 14z + 6t = 9 $$
$$ y = \frac{9 + 14z - 6t}{3} $$

6. Podstawiamy $ y $ do pierwszego równania:
$$ 2\left(\frac{9 + 14z - 6t}{3}\right) - 4z + t = 3 $$
$$ \frac{18 + 28z - 12t}{3} - 4z + t = 3 $$
$$ 18 + 28z - 12t - 12z + 3t = 9 $$
$$ 16z - 9t = -9 $$
$$ z = \frac{-9 + 9t}{16} $$

7. Podstawiamy $ z $ do wyrażeń na $ x $ i $ y $:
$$ x = \frac{5 - 8\left(\frac{-9 + 9t}{16}\right) + 9t}{6} $$
$$ x = \frac{5 - \frac{-72 + 72t}{16} + 9t}{6} $$
$$ x = \frac{5 - \frac{-72}{16} - \frac{72t}{16} + 9t}{6} $$
$$ x = \frac{5 - \frac{-36}{8} - \frac{36t}{8} + 9t}{6} $$
$$ x = \frac{5 - \frac{-18}{4} - \frac{18t}{4} + 9t}{6} $$

$$ y = \frac{8 + 10\left(\frac{-9 + 9t}{16}\right) - 18t}{6} $$
$$ y = \frac{8 + \frac{-90 + 90t}{16} - 18t}{6} $$
$$ y = \frac{8 + \frac{-90}{16} + \frac{90t}{16} - 18t}{6} $$
$$ y = \frac{8 + \frac{-45}{8} + \frac{45t}{8} - 18t}{6} $$

Rozwiązanie:
$$ x = \frac{5 - \frac{-18}{4} - \frac{18t}{4} + 9t}{6}, \quad y = \frac{8 + \frac{-45}{8} + \frac{45t}{8} - 18t}{6}, \quad z = \frac{-9 + 9t}{16} $$